In [9]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("alexistubulekasA3_partA")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

In [10]:
rdd = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.en',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
    conf={'textinputformat.record.delimiter': '\n'}
)\
.cache() # Keep this RDD in memory!

rdd.count()
# Only one job (previous .cache() did not trigger a job)

1862234

In [11]:
rdd.getNumPartitions()

2

In [12]:
rdd1 = spark_context.newAPIHadoopFile(
    'hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.sv',
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
    'org.apache.hadoop.io.LongWritable',
    'org.apache.hadoop.io.Text',
    conf={'textinputformat.record.delimiter': '\n'}
)\
.cache() # Keep this RDD in memory!

rdd.count()
# Only one job (previous .cache() did not trigger a job)

1862234

In [13]:
rdd1.getNumPartitions()

3

In [19]:
rdd_take3 = rdd.take(3)
print(rdd_take3)
# [(line_number, partition)]

[(0, 'Resumption of the session'), (26, 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'), (234, "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.")]


In [30]:
def lowercase_and_split(rdd_input):
    """Takes in a rdd and outputs all the values in second position in the tuple
    lowercased and split on space"""
    return rdd_input.map(lambda w: w[1].lower().split(' '))

rdd_lower_split = lowercase_and_split(rdd)
rdd1_lower_split = lowercase_and_split(rdd1)

In [26]:
#rdd_lower = rdd.map(lambda w: w[1].lower().split(' '))


In [31]:
# release the cores for another application!
spark_context.stop()